# BIBNet - Map Library Locations
Using the BIBNet API (see https://data.bnl.lu/apis/infobib/), this notebook will get the list of all libraries in Luxembourg.
For each library, it will retrieve detailed information including address and opening hours.
The library will then be plotted on an OpenStreetMap of Luxembourg.

Required projects:
Folium: https://python-visualization.github.io/folium/latest/index.html

In [22]:
import requests
import json
import html
import time
%pip install folium
import folium

Note: you may need to restart the kernel to use updated packages.


In [23]:
# set up the referer and the user-agent as required by Nominatum User Policy
# https://operations.osmfoundation.org/policies/nominatim/
bnl_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'referer': 'https://data.bnl.lu/'
}

In [24]:
# Retrieve the list of libraries in json format
# this provides only the name and the aleph sub lib code
api_url = "https://infobib.bibnet.lu/bibnet-libraries.json"
response = requests.get(api_url)

In [25]:
# convert the list of libraries to the Python dict format
libraries = response.json()

In [26]:
library_data = {}

# for each library in the list, search for further details include address
for library_list in libraries["libraries"]:
    lib_code = library_list["alephSublibCode"]
    lib_name = library_list["libraryName"]
    
    # build the url with the code for the library
    lib_url = "https://infobib.bibnet.lu/show.php?lib=" + lib_code + "&format=json&lang=eng"
    try:
        response = requests.get(lib_url)
    except Exception:
        # If there is an error with the request, we skip this library
        continue
        
    lib_details = response.json()
    
    # if there is an address (not all libraries have one) 
    # then build the search criteria to get the longitude and latitude
    # otherwise skip this entry
    if ("address" in lib_details):
        # Get separate street, city and post code which are HTML encoded
        lib_street = html.unescape(lib_details["address"]["address"])
        lib_city = html.unescape(lib_details["address"]["place"])
        lib_postcode = lib_details["address"]["postcode"]
        
        # Build the search query to get longitude and latitue
        coord_url = "https://nominatim.openstreetmap.org/search.php?format=jsonv2&accept-language=en-gb&" \
            + "street=" + html.escape(lib_street) + "&city=" + html.escape(lib_city) \
            + "&country=Luxembourg&postalcode=" + lib_postcode[2:]

        # request the longitude and latitude from OpenStreetMap api
        response = requests.get(coord_url, headers=bnl_headers)
        coord_details = response.json()

        # if there are results, then get the longitude and latitude
        # otherwise skip this entry
        if (len(coord_details) >0 ):
            latitude = coord_details[0]["lat"]
            longitude = coord_details[0]["lon"]
        else:
            continue
    else:
        continue
        
    # add the library to the list with it's address, latitude and longitude using the library name as the key
    library_data[lib_name] = lib_street + ", " + lib_city + " " + lib_postcode, latitude, longitude

    # wait 1 second as we cannot send more than 1 request per second to Nominatum
    # https://operations.osmfoundation.org/policies/nominatim/
    time.sleep(1)

## Using Folium
Folium is used to easily display an OpenStreetMap map. 
Documentation is available here:
https://python-visualization.github.io/folium/latest/index.html

In [27]:
# use Colmar-Berg as centre point of map so the display shows all the points
# display the scale and allow zooming
map = folium.Map(location=[49.8115688,6.0013008], zoom_start=9, control_scale=True, zoom_control=True)

# add each library with it's name and address as the popup so that it can be copied
for library_name, library_coord in library_data.items():
    folium.Marker([library_coord[1], library_coord[2]], popup=library_name + "\n" + library_coord[0]).add_to(map)       
map